**Create a pipeline with a text-generation task**

In [ ]:
import torch
from transformers import AutoTokenizer,pipeline
# model_id = "togethercomputer/Llama-2-7B-32K-Instruct"
model_id = "NousResearch/Llama-2-7b-chat-hf"


tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

**Test Model with a mathematical example.**

In [ ]:
prompt_template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Return final results in numeric format.
[/INST]
""".strip()


sequences = pipeline(
    prompt_template,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Result: [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Return final results in numeric format.
[/INST]
Janet makes $2 x 12 = $24 per day at the farmers' market.


**Test Model with a mathematical example using zero-shot in-context learning method**

In [ ]:
prompt_template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Let's think step by step then return final results in numeric format.
[/INST]
""".strip()


sequences = pipeline(
    prompt_template,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Let's think step by step then return final results in numeric format.
[/INST]  Sure, I'd be happy to help you with that! Here's the calculation:

Janet lays 16 eggs per day.
She eats 3 eggs for breakfast every day, so 16 - 3 = 13 eggs left.
She bakes muffins with 4 eggs, so 13 - 4 = 9 eggs left.
She sells the remaining eggs at the farmers' market for $2 per egg.
So, Janet makes $2 x 9 = $18 per day


**Error of Low GPU Memomry**



In [ ]:
prompt_template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
[/INST]
""".strip()


sequences = pipeline(
    prompt_template,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=2,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)

for seq in sequences:
    print("*****"*50)
    print(f"Result: {seq['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


**Install bitsandbytes library to quantize a model.**

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00


**Create a pipeline with text-generation task and 4-bit quantized model**

In [ ]:
import torch
from transformers import AutoTokenizer,pipeline
model_id = "NousResearch/Llama-2-7b-chat-hf"


tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    model_kwargs={"load_in_4bit": True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

**Test 4-bit quantized model**

In [ ]:
prompt_template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
[/INST]
""".strip()


sequences = pipeline(
    prompt_template,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=256,
)

for seq in sequences:
    print("*****"*50)
    print(f"Result: {seq['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


**********************************************************************************************************************************************************************************************************************************************************
Result: [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
[/INST]  $8 per day.


**Create a pipeline with text-generation task and a small 1.1B model**

In [ ]:
import torch
from transformers import AutoTokenizer,pipeline
model_id = "TinyLlama/TinyLlama-1.1B-step-50K-105b"


tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
    )

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
prompt_template = """[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
[/INST]
""".strip()


sequences = pipeline(
    prompt_template,
    do_sample=True,
    top_k=10,
    temperature=0.7,
    top_p=0.95,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
    max_length=250
    )

for seq in sequences:
    print("*****"*50)
    print(f"Result: {seq['generated_text']}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


**********************************************************************************************************************************************************************************************************************************************************
Result: [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Answer exactly in few words.
<</SYS>>

Answer the question below:

Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
[/INST]


A: I would use the following code to answer your question:


*

*Firstly, use the function get_total_daily_economy() to get the total amount of money spent by each user for the week.

*Then, use the following function to get the total amount of money for the day.


$total_daily_economy = 0;

$total_daily_economy = 